# Note

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
- 문제 유형 별 정답이 포함된 데이터는 context에 있음.
- 문제 유형 별 정답은 context 중 정답인 토큰의 위치를 나타냄. (answer_start, answer_end) 또한 text에는 정답의 위치 대신 정수 인코딩 전의 정답(인코딩 전) 들어가있음
- 다지선다형의 경우 text 또는 clue_text에 [번호. 정답] 형태로 기재되어 있음.
- Table 정답 추출형의 경우 table에 토큰화된 형태로 테이블이 저장되어 있음.
"""

"""
1. context를 system prompt + 프롬프트 직접 설정 / question은 그대로 / answer은 "text"로 할건지 or answer_start, answer_end로 할건지 결정해야함
2. Table 정답 추출형의 경우 아예 제거 or table에 대한 정답은 answer_start, answer_end에 있으니까 table을 system_prompt로 설정할지
"""

'\n1. context를 system prompt + 프롬프트 직접 설정 / question은 그대로 / answer은 "text"로 할건지 or answer_start, answer_end로 할건지 결정해야함\n2. Table 정답 추출형의 경우 아예 제거 or table에 대한 정답은 answer_start, answer_end에 있으니까 table을 system_prompt로 설정할지\n'

# Install Library

In [ ]:
!pip install torch==2.4.0 transformers==4.45.1 datasets==3.0.1 accelerate==0.34.2 trl==0.11.1 peft==0.13.0

ERROR: Operation cancelled by user


In [ ]:
import torch
import json

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

# Data Load

In [ ]:
# Train data
with open("/content/drive/MyDrive/데이콘/train_data.json", "r", encoding = "utf-8") as f:
    train_data = json.load(f)

In [ ]:
print(len(train_data))

112143


In [ ]:
train_data[0:10]

[{'doc_id': 'TNL041227',
  'doc_title': '신 미·일방위협력지침의 주요내용 및 쟁점과 한국의 대응과제',
  'doc_source': '국회입법조사처',
  'doc_published': 20150603,
  'created': '20221219143503',
  'doc_class': {'class': '법률/사회과학/법학일반', 'code': '연구보고서'},
  'paragraphs': [{'context_id': '2356574',
    'context': '1. 들어가며\n지난 2015년 4월 27일 아베신조(安倍晋三) 총리의 방미를 계기로 미·일 양국은 ‘신 미·일방위협력지침’에 합의하였다. 이는 미·일 간의 안보협력관계를 체계적으로 실행하기 위해 1978년에 마련되고, 1997년에 개정된 ‘미·일방위협력지침’을 대체하는 것으로 미·일 동맹의 군사적 활동 공간을 전 세계로 확장하는 내용을 담고 있다.\n이에 대해서 중국은 자국의 이익을 훼손할 수 있다는 점에서 비판적인 입장을 피력했는데, 실제로 아베 총리는 미·일 동맹 강화가 북한뿐만 아니라 중국의 위협에 대응하기 위한 것이라고 발언함으로써 동북아의 평화안보질서에 부정적인 영향을 미치게 될 것이라는 주장에 힘이 실리고 있다.',
    'tbs': None,
    'qas': [{'qa_type': 1,
      'question_id': '6379347',
      'question': '처음으로 미·일방위협력지침이 정해진 건 언제야',
      'answer': {'source': '2356574',
       'text': '1978년',
       'answer_start': 109,
       'answer_end': 113,
       'cell_text': None,
       'cell_coordinates': None,
       'clue_start': None,
       'clue_text': None,
       'opti

In [ ]:
# Valid data
with open("/content/drive/MyDrive/데이콘/valid_data.json", "r", encoding = "utf-8") as f:
    valid_data = json.load(f)

In [ ]:
print(len(valid_data))

14018


In [ ]:
valid_data[0]

{'doc_id': 'TNL042650',
 'doc_title': '고령사회를 대비한 고령운전자의 교통안전대책(인구구조 변화에 대한 정책대응 연구2호)',
 'doc_source': '국회입법조사처',
 'doc_published': 20100611,
 'created': '20221219144746',
 'doc_class': {'class': '법률/사회과학/노동·법률정책', 'code': '연구보고서'},
 'paragraphs': [{'context_id': '2718229',
   'context': '○ [표 5]는 장래 연령별 운전자수를 예측한 결과인데, 이에 따르면 65세 이상 운전자는 2010년에 106만명(65세 이상 인구 중 21.1%)에서 2020년에는 233만명(65세 이상 인구 중 33.8%)으로 증가할 것이라고 예측함<br>(단위:천명)',
   'tbs': [{'table_id': '2718229',
     'table_title': '[표 5] 장래 연령별 운전자수 예측',
     'table': '<table><tbody><tr><td>구분</td><td>1999년</td><td>2005년</td><td>2010년</td><td>2015년</td><td>2020년</td></tr><tr><td>18~20세</td><td>514</td><td>692</td><td>824</td><td>962</td><td>1,079</td></tr><tr><td>21~30세</td><td>4,976</td><td>5,450</td><td>5,500</td><td>5,644</td><td>6,024</td></tr><tr><td>31~40세</td><td>5,969</td><td>6,036</td><td>5,971</td><td>5,994</td><td>6,050</td></tr><tr><td>41~50세</td><td>3,813</td><td>6,017</td><td>6,029</td><td>6,097</td><td>6,031</td></t

# Chat template

In [ ]:
def chat_template(document : dict) -> list:
    """
    주어진 문서(JSON 한 건)에서 QA 쌍을 기반으로 통합 챗 템플릿 형식의 데이터 생성
    - 문단 기반(qa_type = 1)
    - 표 기반(qa_type = 3)

    Returns : list of dict (챗 템플릿 형태)
    """

    result = []

    doc_title = document.get("doc_title", "")
    doc_source = document.get("doc_source", "")
    doc_published = str(document.get("doc_published", ""))
    paragraphs = document.get("paragraphs", [])

    for para in paragraphs:
        context = para.get("context", "")
        qas = para.get("qas", [])
        table_info = para.get("tbs", [])

        # 표 정보 전처리
        table_map = {}
        if table_info:
            try:
                html_table = table_info[0]["table"]
                soup = BeautifulSoup(html.unescape(html_table), "html.parser")
                rows = soup.find_all("tr")

                for i, row in enumerate(rows):
                    cols = row.find_all(["td", "th"])
                    for j, col in enumerate(cols):
                        table_map[(i, j)] = col.get_text(strip = True)

            except:
                table_map = {}

        for qa in qas:
            qa_type = qa.get("qa_type", 1)
            question = qa.get("question", "")
            answer = qa.get("answer", {}).get("text", "")

            template = {
            "user_question" : question,
            "document_title" : doc_title,
            "document_source" : doc_source,
            "document_published" : doc_published,
            "context" : context,
            "answer" : answer
            }

            if qa_type == 3 and qa.get("answer", {}).get("cell_coordinates"):
                try:
                    coord = qa["answer"]["cell_coordinates"][0]
                    row_idx, col_idx = eval(coord)
                    row_label = table_map.get((row_idx, 0), "")
                    col_label = table_map.get((0, col_idx), "")
                    cell_value = table_map.get((row_idx, col_idx), answer)

                    template["cell_info"] = {
                        "row_label" : row_label,
                        "column_label" : col_label,
                        "cell_value" : cell_value
                    }
                except:
                    template["cell_info"] = {
                        "row_label" : "",
                        "column_label" : "",
                        "cell_value" : answer
                    }

            result.append(template)
    return result

In [ ]:
# 모든 문서에 적용
all_templates = []

total_qas = 0
valid_qas = 0

for doc in train_data:

    for para in doc.get("paragraphs", []):
        qas = para.get("qas", [])
        total_qas += len(qas)
        valid_qas += len(chat_template(doc))

    all_templates.extend(chat_template(doc))

print(f"총 QA 수: {total_qas}, 변환된 템플릿 수: {valid_qas}")

총 QA 수: 316416, 변환된 템플릿 수: 316416


In [ ]:
# 모든 문서에 적용
all_templates = []

total_qas = 0
valid_qas = 0

for doc in valid_data:

    for para in doc.get("paragraphs", []):
        qas = para.get("qas", [])
        total_qas += len(qas)
        valid_qas += len(chat_template(doc))

    all_templates.extend(chat_template(doc))

print(f"총 QA 수: {total_qas}, 변환된 템플릿 수: {valid_qas}")

총 QA 수: 42190, 변환된 템플릿 수: 42190
